In [1]:
import os, sys
import logging
from datetime import datetime
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logger = logging.getLogger(__name__)
logger.setLevel(logging.NOTSET)

logger.info("Application started successfully.")


2025-06-30 07:31:12,086 - INFO - Application started successfully.


In [2]:
# load environment variables

storage_account = os.environ["AZURE_STORAGE_ACCOUNT_NAME"]
access_key = os.environ["AZURE_STORAGE_ACCOUNT_ACCESS_KEY"]
NESSIE_URI = os.environ['NESSIE_URI']
REF = "etl"
FULL_PATH_TO_WAREHOUSE = os.environ['WAREHOUSE']
AWS_S3_ENDPOINT = os.environ["AWS_S3_ENDPOINT"]
AWS_ACCESS_KEY = os.environ['AWS_ACCESS_KEY_ID']
AWS_SECRET_KEY  = os.environ['AWS_SECRET_ACCESS_KEY']

In [3]:
# set pyspark configuration
from pyspark import SparkConf

conf = SparkConf() \
    .setAppName("Read_csv_from_azure_blob") \
    .setMaster("local[*]") \
    .set("spark.executor.memory", "4g") \
    .set("spark.driver.memory", "4g")

# conf.set("spark.sql.shuffle.partitions", 200)
# conf.set("spark.sql.shuffle.autoBroadcastJoinThreshold", 1)
# conf.set("spark.spark.executor.extraJavaOptions", "-XX:+UseG1GC")
# conf.set("spark.spark.driver.extraJavaOptions", "-XX:+UseG1GC")




In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

25/06/30 07:31:13 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [6]:
# create namespace gold
namespace = "gold"
spark.sql(f"CREATE NAMESPACE IF NOT EXISTS {namespace}")
spark.sql(f"SHOW NAMESPACES IN nessie").show()

+---------+
|namespace|
+---------+
|   silver|
|   bronze|
|     gold|
+---------+



In [7]:
# create etl branch
spark.sql(f"USE REFERENCE main IN nessie")
spark.sql(f"DROP BRANCH IF EXISTS {REF}")
spark.sql(f"CREATE BRANCH {REF} IN nessie FROM main")
spark.sql(f"LIST REFERENCES IN nessie").show()


+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch| etl|4f60fe246bc1dfcc7...|
| Branch|main|4f60fe246bc1dfcc7...|
+-------+----+--------------------+



In [8]:
import notebooks.lib.gold_fact_sales_sql as sql
import importlib
importlib.reload(sql)

source_list = [
    {
        "table_name": "INTERNET_SALES_HEADER",
        "sql_create_table": sql.sql_create_gold_internet_sales_header,
        "sql_merge": sql.sql_merge_gold_internet_sales_header,
        "enabled": True
    },
    {
        "table_name": "INTERNET_SALES_DETAIL",
        "sql_create_table": sql.sql_create_gold_internet_sales_detail,
        "sql_merge": sql.sql_merge_gold_internet_sales_detail,
        "enabled": True
    }
]

In [9]:
# change branch to etl
spark.sql(f"USE REFERENCE main IN nessie").show()

for item in [item for item in source_list if item["enabled"]]:
    table_name = item["table_name"].upper()

    # drop table
    # logger.info(f'dropping table if exists: {namespace}.{table_name}')
    # spark.sql(f'DROP TABLE IF EXISTS {namespace}.{table_name}')
    
    # create table
    logger.info(f'creating table if not exists:{table_name}')
    spark.sql(item["sql_create_table"])

    # merge data into gold
    try:
        logger.info(f'merging into: {table_name}')
        spark.sql(item["sql_merge"])
        logger.info(f'data merged successfully')
    except Exception as e:
        print(str(e))



2025-06-30 07:31:19,411 - INFO - creating table if not exists:INTERNET_SALES_HEADER


+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch|main|4f60fe246bc1dfcc7...|
+-------+----+--------------------+



25/06/30 07:31:19 WARN S3FileIO: Unclosed S3FileIO instance created by:
	org.apache.iceberg.aws.s3.S3FileIO.initialize(S3FileIO.java:444)
	org.apache.iceberg.CatalogUtil.loadFileIO(CatalogUtil.java:402)
	org.apache.iceberg.CatalogUtil.loadFileIO(CatalogUtil.java:349)
	org.apache.iceberg.nessie.NessieCatalog.initialize(NessieCatalog.java:132)
	org.apache.iceberg.CatalogUtil.loadCatalog(CatalogUtil.java:277)
	org.apache.iceberg.CatalogUtil.buildIcebergCatalog(CatalogUtil.java:331)
	org.apache.iceberg.spark.SparkCatalog.buildIcebergCatalog(SparkCatalog.java:153)
	org.apache.iceberg.spark.SparkCatalog.initialize(SparkCatalog.java:752)
	org.apache.spark.sql.execution.datasources.v2.NessieCatalogBridge.setCurrentRefForSpark(NessieCatalogBridge.java:105)
	org.apache.spark.sql.execution.datasources.v2.UseReferenceExec.runInternal(UseReferenceExec.scala:44)
	org.apache.spark.sql.execution.datasources.v2.NessieExec.run(NessieExec.scala:34)
	org.apache.spark.sql.execution.datasources.v2.V2Command

In [10]:
spark.sql(f"MERGE BRANCH {REF} INTO main IN nessie").show()
spark.sql(f"DROP BRANCH {REF} IN nessie").show()

+----+--------------------+
|name|                hash|
+----+--------------------+
|main|b1768c93d7afb584f...|
+----+--------------------+

+------+
|status|
+------+
|    OK|
+------+



In [11]:
spark.sql("LIST REFERENCES IN nessie").show()

+-------+----+--------------------+
|refType|name|                hash|
+-------+----+--------------------+
| Branch|main|b1768c93d7afb584f...|
+-------+----+--------------------+

